In [11]:
# ids of games in games/ folder
game_ids = ['0042100301','0042100304','0042100307','0042100313','0042100401','0042100404','0042100302','0042100305','0042100311','0042100314','0042100402','0042100405','0042100303','0042100306','0042100312','0042100315','0042100403','0042100406']

import json
import pandas as pd
from py_ball import playbyplay

#### Pull PBP ####
# Header information to pass along to the stats.nba.com API to make it play nicely
HEADERS = {'Connection': 'keep-alive',
           'Host': 'stats.nba.com',
           'Origin': 'http://stats.nba.com',
           'Upgrade-Insecure-Requests': '1',
           'Referer': 'stats.nba.com',
           'x-nba-stats-origin': 'stats',
           'x-nba-stats-token': 'true',
           'Accept-Language': 'en-US,en;q=0.9',
           "X-NewRelic-ID": "VQECWF5UChAHUlNTBwgBVw==",
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6)' +\
                         ' AppleWebKit/537.36 (KHTML, like Gecko)' + \
                         ' Chrome/81.0.4044.129 Safari/537.36'}

game_id = '0042100301'
plays = playbyplay.PlayByPlay(headers=HEADERS,
                              endpoint='playbyplayv2',
                              game_id=game_id)

play_df = pd.DataFrame(plays.data['PlayByPlay'])

#### Load in event data ####
# Assuming the event files are stored in a "tracking_data" subdirectory
file_path = f"./games/{game_id}/{game_id}_events.jsonl"

# Open .jsonl
with open(file_path, 'r') as f:
    json_list = list(f)

# Unpack .jsonl file
event_list = []
for json_str in json_list:
    result = json.loads(json_str)
    event_list.append(result)

# Create an event DataFrame
event_df = pd.DataFrame(event_list)
event_df.head()

,gameId,eventType,shotClock,gameClock,wallClock,period,homePlayers,awayPlayers,playerId,pbpId
0,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,TOUCH,24.0,719.76,1652834779899,1,"[82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b, 267de25...","[297a0fda-0a03-4003-bf39-aa92b7a730ff, 986b713...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN
1,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,DRIBBLE,24.0,719.76,1652834779899,1,"[82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b, 267de25...","[297a0fda-0a03-4003-bf39-aa92b7a730ff, 986b713...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN
2,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,PASS,24.0,719.04,1652834780619,1,"[82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b, 267de25...","[297a0fda-0a03-4003-bf39-aa92b7a730ff, 986b713...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN
3,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,TOUCH,23.9,718.96,1652834780699,1,"[82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b, 267de25...","[297a0fda-0a03-4003-bf39-aa92b7a730ff, 986b713...",297a0fda-0a03-4003-bf39-aa92b7a730ff,NaN
4,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,DRIBBLE,23.2,718.28,1652834781379,1,"[82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b, 267de25...","[297a0fda-0a03-4003-bf39-aa92b7a730ff, 986b713...",297a0fda-0a03-4003-bf39-aa92b7a730ff,NaN


In [12]:
print(f'playdf {len(play_df.index)}')
print(f'eventdf {len(event_df.index)}')
play_df.head()
play_df.to_csv('temp-pbp.csv')

playdf 458
eventdf 3610


In [13]:
# Join events to play-by-play
# Some events occur in between discrete play-by-play, so we'll do a left join here
joint_df = event_df.merge(play_df, left_on="pbpId", right_on="EVENTNUM", how="left")

# And you can check out some events mapped to a play-by-play event
# Here's a pass from Jayson Tatum to Grant Williams for a successful three-point FG attempt
joint_df[joint_df["pbpId"]==7][["eventType", "shotClock", "gameClock", "VISITORDESCRIPTION", "PCTIMESTRING"]]

,eventType,shotClock,gameClock,VISITORDESCRIPTION,PCTIMESTRING
24,PASS,8.2,703.24,Williams 25' 3PT Step Back Jump Shot (3 PTS) (...,11:39
27,SHOT,5.9,700.96,Williams 25' 3PT Step Back Jump Shot (3 PTS) (...,11:39


In [15]:
joint_df.head()
joint_df.to_csv('joint.csv')